In [ ]:
import pandas as pd
import glob
import requests
import re
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm


def main():
    # Part 1: Merge Title/DOI pairs from all group CSV files
    dfs = []
    title_candidates = ["ProjectTitle", "Title", "title"]
    doi_candidates = ["DOI", "doi"]
    for path in glob.glob("group*.csv"):
        df = pd.read_csv(path, dtype=str)
        title_col = next((c for c in title_candidates if c in df.columns), None)
        doi_col = next((c for c in doi_candidates if c in df.columns), None)
        if doi_col is None:
            print(f"Skipping {path}: DOI column not found")
            continue
        sub = pd.DataFrame({
            "Title": df[title_col] if title_col else pd.NA,
            "DOI": df[doi_col]
        }).dropna(subset=["DOI", "Title"])
        dfs.append(sub)

    if dfs:
        combined = pd.concat(dfs, ignore_index=True).drop_duplicates(subset=["DOI", "Title"])  # Deduplicate
    else:
        combined = pd.DataFrame(columns=["Title", "DOI"])

    combined.to_csv("merged_titles_dois.csv", index=False)
    print(f"Merged {combined.shape[0]} records")

    # Part 2: Enrich with project metadata
    df = pd.read_csv("merged_titles_dois.csv", dtype=str)
    meta = pd.read_excel("ProjectsAllMetadata.xlsx", sheet_name="All Metadata", dtype=str)
    meta_sub = meta[["Proj ID", "Status", "Title", "RDC", "Start Year", "End Year", "PI"]].rename(columns={
        "Proj ID": "ProjID",
        "Status": "ProjectStatus",
        "Title": "Title_meta",
        "RDC": "ProjectRDC",
        "Start Year": "ProjectYearStarted",
        "End Year": "ProjectYearEnded",
        "PI": "ProjectPI"
    })
    enriched = df.merge(
        meta_sub,
        left_on="Title",
        right_on="Title_meta",
        how="left"
    )
    final_cols = [
        "Title", "DOI",
        "ProjID", "ProjectStatus", "ProjectRDC",
        "ProjectYearStarted", "ProjectYearEnded", "ProjectPI"
    ]
    result = enriched[final_cols].copy()
    result["DOI"] = result["DOI"].str.strip().str.replace(
        r"^https?://(?:dx\.)?doi\.org/", "", regex=True
    )
    result.to_csv("merged_with_project_metadata.csv", index=False)
    print(f"Total records: {len(result)}, Projects matched: {result['ProjID'].notna().sum()}")

    # Part 3: Fetch metadata from Crossref and OpenAlex
    df = pd.read_csv("merged_with_project_metadata.csv", dtype=str)

    def fetch_crossref(doi: str) -> dict:
        url = f"https://api.crossref.org/works/{doi}"
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()
        data = resp.json()["message"]
        title = (data.get("title") or [""])[0]
        authors = data.get("author", [])
        author_strs = [f"{a.get('family','')}, {a.get('given','')}" for a in authors]
        biblio = "; ".join(author_strs) + \
            f" ({data.get('issued',{}).get('date-parts',[[None]])[0][0]}). {title}. " \
            f"{(data.get('container-title') or [''])[0]}"
        type_map = {
            "journal-article": "JA",
            "book-chapter": "BC",
            "dissertation": "DI",
            "report": "RE",
            "posted-content": "WP"
        }
        out_type = type_map.get(data.get("type", ""), "")
        out_status = "PB" if data.get("status", "") == "published" else "FC"
        issued = data.get("issued", {}).get("date-parts", [[None, None]])[0]
        return {
            "OutputTitle": title,
            "OutputBiblio": biblio,
            "OutputType": out_type,
            "OutputStatus": out_status,
            "OutputVenue": (data.get("container-title") or [""])[0],
            "OutputYear": issued[0],
            "OutputMonth": issued[1] if len(issued) > 1 else None,
            "OutputVolume": data.get("volume", ""),
            "OutputNumber": data.get("issue", ""),
            "OutputPages": data.get("page", "")
        }

    def fetch_openalex(doi: str) -> dict:
        url = f"https://api.openalex.org/works/{doi}"
        resp = requests.get(url, timeout=10)
        if resp.status_code != 200:
            return {}
        data = resp.json().get("data", {})
        auths = [a["author"]["display_name"] for a in data.get("authorships", [])]
        return {"Authors": ", ".join(auths)} if auths else {}

    def scrape_all_metadata(doi: str) -> dict:
        cr_meta = fetch_crossref(doi)
        oa_meta = fetch_openalex(doi)
        if oa_meta.get("Authors"):
            cr_meta["Authors"] = oa_meta["Authors"]
        return cr_meta

    dois = df["DOI"].dropna().unique().tolist()
    records = []
    for doi in dois:
        try:
            print(f"Fetching metadata for DOI: {doi}")
            meta = scrape_all_metadata(doi)
            meta["DOI"] = doi
            records.append(meta)
        except Exception as e:
            print(f"Error fetching DOI={doi}: {e}")

    meta_df = pd.DataFrame.from_records(records)
    cols = ["DOI"] + [c for c in meta_df.columns if c != "DOI"]
    meta_df = meta_df[cols]
    merged = df.merge(meta_df, on="DOI", how="left")
    merged.to_csv("with_all_Output_metadata.csv", index=False)
    print("Saved intermediate metadata to: with_all_Output_metadata.csv")
    merged = pd.read_csv("with_all_Output_metadata.csv")
    merged = merged.dropna(subset=["OutputBiblio"])
    merged.to_csv("with_all_Output_metadata_cleaned.csv", index=False)
    # Part 4: Clean and filter records
    df_clean = pd.read_csv("with_all_Output_metadata_cleaned.csv", dtype=str)
    mask = pd.Series(True, index=df_clean.index)
    mask &= df_clean["OutputPages"].apply(
        lambda x: bool(re.fullmatch(r"\d+(-\d+)?", x.strip()))
        if isinstance(x, str) and x.strip() else True
    )
    for col in df_clean.columns:
        df_clean[col] = df_clean[col].astype(str)
        mask &= ~df_clean[col].apply(
            lambda txt: bool(re.search(r"[\u4e00-\u9fff]", txt))
        )
    mask &= (df_clean.shape[1] <= 18)
    df_final = df_clean[mask]
    df_final.to_csv("with_all_Output_metadata_final.csv", index=False)

    # Part 5: Fetch authors again for final set
    df_auth = pd.read_csv("with_all_Output_metadata_final.csv", dtype=str)
    df_auth['author'] = df_auth['DOI'].apply(lambda d: fetch_crossref(d).get('OutputBiblio', ''))
    df_auth.to_csv("with_all_Output_metadata_with_authors.csv", index=False)
    print("Authors added to final set and saved to: with_all_Output_metadata_with_authors.csv")

    # Part 6: Match researchers
    df_authors = pd.read_csv("with_all_Output_metadata_with_authors.csv", dtype=str)
    df_meta_r = pd.read_excel("ProjectsAllMetadata.xlsx", sheet_name="Researchers", dtype=str)
    researchers = df_meta_r[['Researcher','Proj ID','Status','Title','RDC','Start Year','End Year','PI']]

    def match_researchers_to_project(author_str):
        if not isinstance(author_str, str) or not author_str.strip():
            return []
        matched = []
        for name in author_str.split('; '):
            mask = researchers['Researcher'].str.lower().str.find(name.lower()) >= 0
            subset = researchers[mask]
            if not subset.empty:
                matched.append(subset.iloc[0])
        return matched

    with ThreadPoolExecutor(max_workers=10) as executor:
        match_data = list(tqdm(
            executor.map(match_researchers_to_project, df_authors['author']),
            total=len(df_authors), desc="Matching researchers"
        ))

    for col in ['Proj ID','Status','Title','RDC','Start Year','End Year','PI']:
        df_authors[col] = [
            ", ".join(str(r[col]) for r in rows) if rows else ""
            for rows in match_data
        ]
    df_authors.to_csv("with_all_Output_metadata_with_authors_and_projects.csv", index=False)
    print("Researcher fields appended and saved to: with_all_Output_metadata_with_authors_and_projects.csv")

    # Part 7: Final cleanup
    df_out = pd.read_csv("with_all_Output_metadata_with_authors_and_projects.csv", dtype=str)
    drop_cols = ['Title','ProjID','ProjectStatus','ProjectRDC','ProjectYearEnded','ProjectPI']
    df_out.drop(columns=drop_cols, inplace=True)
    df_out.dropna(subset=['Proj ID'], inplace=True)
    df_out.to_csv("processed_data.csv", index=False)
    df_out = pd.read_csv("processed_data.csv", dtype=str)
    df_out.dropna(subset=['Proj ID'], inplace=True)
    df_out.drop(columns=['ProjectYearStarted'], inplace=True)
    df_out.to_csv("processed_data_cleaned.csv", index=False)
    df_out.to_excel("processed_data_cleaned.xlsx", index=False)
    print(df_out.head())

if __name__ == "__main__":
    main()


In [ ]:
def fetch_crossref(doi: str) -> dict:
    url = f"https://api.crossref.org/works/{doi}"
    resp = requests.get(url, timeout=20)
    resp.raise_for_status()
    data = resp.json()["message"]
    title = (data.get("title") or [""])[0]
    authors = data.get("author", [])
    author_strs = [f"{a.get('family','')}, {a.get('given','')}" for a in authors]
    biblio = "; ".join(author_strs) + \
        f" ({data.get('issued',{}).get('date-parts',[[None]])[0][0]}). {title}. " \
        f"{(data.get('container-title') or [''])[0]}"
    type_map = {
        "journal-article": "JA",
        "book-chapter": "BC",
        "dissertation": "DI",
        "report": "RE",
        "posted-content": "WP"
    }
    out_type = type_map.get(data.get("type", ""), "")
    out_status = "PB" if data.get("status", "") == "published" else "FC"
    issued = data.get("issued", {}).get("date-parts", [[None, None]])[0]
    return {
        "OutputTitle": title,
        "OutputBiblio": biblio,
        "OutputType": out_type,
        "OutputStatus": out_status,
        "OutputVenue": (data.get("container-title") or [""])[0],
        "OutputYear": issued[0],
        "OutputMonth": issued[1] if len(issued) > 1 else None,
        "OutputVolume": data.get("volume", ""),
        "OutputNumber": data.get("issue", ""),
        "OutputPages": data.get("page", "")
    }

def fetch_openalex(doi: str) -> dict:
    url = f"https://api.openalex.org/works/{doi}"
    resp = requests.get(url, timeout=20)
    if resp.status_code != 200:
        return {}
    data = resp.json().get("data", {})
    auths = [a["author"]["display_name"] for a in data.get("authorships", [])]
    return {"Authors": ", ".join(auths)} if auths else {}

def scrape_all_metadata(doi: str) -> dict:
    cr_meta = fetch_crossref(doi)
    oa_meta = fetch_openalex(doi)
    if oa_meta.get("Authors"):
        cr_meta["Authors"] = oa_meta["Authors"]
    return cr_meta

merged = pd.read_csv("with_all_Output_metadata.csv")
merged = merged.dropna(subset=["OutputBiblio"])
merged.to_csv("with_all_Output_metadata_cleaned.csv", index=False)

# Part 4: Clean and filter records
df_clean = pd.read_csv("with_all_Output_metadata_cleaned.csv", dtype=str)
mask = pd.Series(True, index=df_clean.index)
mask &= df_clean["OutputPages"].apply(
    lambda x: bool(re.fullmatch(r"\d+(-\d+)?", x.strip()))
    if isinstance(x, str) and x.strip() else True
)
for col in df_clean.columns:
    df_clean[col] = df_clean[col].astype(str)
    mask &= ~df_clean[col].apply(
        lambda txt: bool(re.search(r"[\u4e00-\u9fff]", txt))
    )
mask &= (df_clean.shape[1] <= 18)
df_final = df_clean[mask]
df_final.to_csv("with_all_Output_metadata_final.csv", index=False)

# Part 5: Fetch authors again for final set
df_auth = pd.read_csv("with_all_Output_metadata_final.csv", dtype=str)
df_auth['author'] = df_auth['DOI'].apply(lambda d: fetch_crossref(d).get('OutputBiblio', ''))
df_auth.to_csv("with_all_Output_metadata_with_authors.csv", index=False)
print("Authors added to final set and saved to: with_all_Output_metadata_with_authors.csv")

# Part 6: Match researchers
df_authors = pd.read_csv("with_all_Output_metadata_with_authors.csv", dtype=str)
df_meta_r = pd.read_excel("ProjectsAllMetadata.xlsx", sheet_name="Researchers", dtype=str)
researchers = df_meta_r[['Researcher','Proj ID','Status','Title','RDC','Start Year','End Year','PI']]

def match_researchers_to_project(author_str):
    if not isinstance(author_str, str) or not author_str.strip():
        return []
    matched = []
    for name in author_str.split('; '):
        mask = researchers['Researcher'].str.lower().str.find(name.lower()) >= 0
        subset = researchers[mask]
        if not subset.empty:
            matched.append(subset.iloc[0])
    return matched

with ThreadPoolExecutor(max_workers=10) as executor:
    match_data = list(tqdm(
        executor.map(match_researchers_to_project, df_authors['author']),
        total=len(df_authors), desc="Matching researchers"
    ))

for col in ['Proj ID','Status','Title','RDC','Start Year','End Year','PI']:
    df_authors[col] = [
        ", ".join(str(r[col]) for r in rows) if rows else ""
        for rows in match_data
    ]
df_authors.to_csv("with_all_Output_metadata_with_authors_and_projects.csv", index=False)
print("Researcher fields appended and saved to: with_all_Output_metadata_with_authors_and_projects.csv")

# Part 7: Final cleanup
df_out = pd.read_csv("with_all_Output_metadata_with_authors_and_projects.csv", dtype=str)
drop_cols = ['Title','ProjID','ProjectStatus','ProjectRDC','ProjectYearEnded','ProjectPI']
df_out.drop(columns=drop_cols, inplace=True)
df_out.dropna(subset=['Proj ID'], inplace=True)
df_out.to_csv("processed_data.csv", index=False)
df_out = pd.read_csv("processed_data.csv", dtype=str)
df_out.dropna(subset=['Proj ID'], inplace=True)
df_out.drop(columns=['ProjectYearStarted'], inplace=True)
df_out.to_csv("processed_data_cleaned.csv", index=False)
df_out.to_excel("processed_data_cleaned.xlsx", index=False)
print(df_out.head())

Authors added to final set and saved to: with_all_Output_metadata_with_authors.csv


Matching researchers: 100%|██████████| 13857/13857 [1:04:00<00:00,  3.61it/s]


Researcher fields appended and saved to: with_all_Output_metadata_with_authors_and_projects.csv
                          DOI  \
0        10.32469/10355/91014   
1              10.17226/22659   
2  10.25300/misq/2017/41:1.03   
3              10.17226/25660   
4    10.1787/9789264308114-en   

                                         OutputTitle  \
0  Bayesian unit-level modeling of non-Gaussian s...   
1               Freight Trip Generation and Land Use   
2  Digital Innovation Management: Reinventing Inn...   
3     Impacts of Policy-Induced Freight Modal Shifts   
4                          Financing Climate Futures   

                                        OutputBiblio OutputType OutputStatus  \
0  , ; Parker, Paul Arthur (None). Bayesian unit-...         DI           FC   
1  Holguín-Veras, José; Jaller, Miguel; Sanchez-D...        NaN           FC   
2  , ; Nambisan, Satish; Lyytinen, Kalle; , ; Maj...         JA           FC   
3  Institute, Rensselaer Polytechnic; , ; , ; , 

In [ ]:
import pandas as pd
import glob
import requests
from datetime import datetime
import re
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def main():
    def fetch_crossref(doi: str) -> dict:
        url = f"https://api.crossref.org/works/{doi}"
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()
        data = resp.json()["message"]
        output_title = data.get("title", [""])[0]
        authors = data.get("author", [])
        author_strs = [f"{a.get('family','')}, {a.get('given','')}" for a in authors]
        output_biblio = "; ".join(author_strs) + \
            f" ({data.get('issued',{}).get('date-parts',[[None]])[0][0]}). " \
            f"{output_title}. {data.get('container-title',[''])[0]}"
        type_map = {
            "journal-article": "JA",
            "book-chapter":     "BC",
            "dissertation":     "DI",
            "report":           "RE",
            "posted-content":   "WP",
        }
        cr_type      = data.get("type", "")
        output_type  = type_map.get(cr_type, "")
        status       = data.get("status","")
        output_status = "PB" if status=="published" else "FC"
        issued = data.get("issued",{}).get("date-parts",[[None,None]])
        return {
            "OutputTitle":   output_title,
            "OutputBiblio":  output_biblio,
            "OutputType":    output_type,
            "OutputStatus":  output_status,
            "OutputVenue":   data.get("container-title",[""])[0],
            "OutputYear":    issued[0][0],
            "OutputMonth":   (issued[0][1] if len(issued[0])>1 else None),
            "OutputVolume":  data.get("volume",""),
            "OutputNumber":  data.get("issue",""),
            "OutputPages":   data.get("page",""),
        }

    def fetch_openalex(doi: str) -> dict:
        url = f"https://api.openalex.org/works/{doi}"
        resp = requests.get(url, timeout=10)
        if resp.status_code != 200:
            return {}
        data = resp.json().get("data",{})
        auths = [a["author"]["display_name"] for a in data.get("authorships",[])]
        return {"Authors": ", ".join(auths)}

    def scrape_all_metadata(doi: str) -> dict:
        meta = fetch_crossref(doi)
        oa   = fetch_openalex(doi)
        if oa.get("Authors"):
            meta["Authors"] = oa["Authors"]
        return meta

    # 2. 定义校验函数
    def valid_year(x):
        # 年份要是 4 位数字，例如 2022
        return bool(isinstance(x, str) and re.fullmatch(r"\d{4}", x.strip()))

    def valid_month(x):
        # 月份为空 OR 1 到 12 之间
        if pd.isna(x) or not x.strip():
            return True
        return bool(re.fullmatch(r"[1-9]|1[0-2]", x.strip()))

    def valid_pages(x):
        # 页码为空 OR “数字” 或 “数字-数字”
        if pd.isna(x) or not x.strip():
            return True
        return bool(re.fullmatch(r"\d+(-\d+)?", x.strip()))

    def valid_doi(x):
        # DOI 格式粗校验：数字.数字/任意
        return bool(isinstance(x, str) and re.fullmatch(r"\d+\.\d+/.+", x.strip()))

    # 3. 定义函数检测中文字符
    def contains_chinese(text):
        """如果文本中包含中文字符，返回 True"""
        if isinstance(text, str):
            return bool(re.search(r'[\u4e00-\u9fff]', text))
        return False

    # 定义一个函数来爬取 DOI 的作者
    def fetch_author_from_doi(doi):
        url = f"https://api.crossref.org/works/{doi}"
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            data = response.json().get('message', {})
            authors = data.get('author', [])

            author_names = []
            for a in authors:
                # 检查是否存在 family 和 given 字段
                family = a.get('family', '')
                given = a.get('given', '')
                if family and given:
                    author_names.append(f"{given} {family}")
                elif family:
                    author_names.append(f"{family}")
                elif given:
                    author_names.append(f"{given}")

            author_str = "; ".join(author_names) if author_names else ""

            # 打印爬取到的作者
            print(f"DOI: {doi} -> Authors: {author_str}")

            return author_str
        except requests.exceptions.RequestException as e:
            print(f"DOI: {doi} -> Error: {e}")
            return ""

    # 1. 定义候选列名
    title_candidates = ["ProjectTitle", "Title", "title"]
    doi_candidates   = ["DOI", "doi"]

    # 2. 读取每个 CSV，提取 Title+DOI 并重命名
    dfs = []
    for path in glob.glob("group*.csv"):
        df = pd.read_csv(path, dtype=str)

        # 找到第一个存在的 Title 列
        title_col = next((c for c in title_candidates if c in df.columns), None)
        # 找到第一个存在的 DOI 列
        doi_col   = next((c for c in doi_candidates   if c in df.columns), None)

        # 如果没有 DOI 列，则跳过这个文件
        if doi_col is None:
            print(f"跳过 {path}，原因：未找到 DOI 列")
            continue

        # 取出并重命名
        sub = pd.DataFrame({
            "Title": df[title_col] if title_col else pd.NA,
            "DOI":   df[doi_col]
        })
        sub = sub.dropna(subset=["DOI", "Title"])
        dfs.append(sub)

    # 3. 合并并去重
    if dfs:
        combined = pd.concat(dfs, ignore_index=True)
        combined = combined.drop_duplicates(subset=["DOI", "Title"], keep="first")
    else:
        combined = pd.DataFrame(columns=["Title","DOI"])

    # 4. 保存结果
    combined.to_csv("merged_titles_dois.csv", index=False)

    # 输出查看
    print(combined.shape[0])

    # 1. 读取只含 Title/DOI 的清洗表
    df = pd.read_csv("merged_titles_dois.csv", dtype=str)

    # 2. 读取项目元数据（All metadata 表）
    meta = pd.read_excel(
        "ProjectsAllMetadata.xlsx",
        sheet_name="All Metadata",
        dtype=str
    )

    # 3. 从 meta 表选出原始列，并重命名以便合并后列名一致
    meta_sub = meta[[
        "Proj ID", "Status", "Title", "RDC", "Start Year", "End Year", "PI"
    ]].rename(columns={
        "Proj ID":       "ProjID",
        "Status":        "ProjectStatus",
        "Title":         "Title_meta",
        "RDC":           "ProjectRDC",
        "Start Year":    "ProjectYearStarted",
        "End Year":      "ProjectYearEnded",
        "PI":            "ProjectPI"
    })

    # 4. 与主表按 Title 做左连接
    enriched = df.merge(
        meta_sub,
        left_on  = "Title",
        right_on = "Title_meta",
        how      = "left"
    )

    # 5. 构建最终表格列序
    final_cols = [
        "Title", "DOI",
        "ProjID", "ProjectStatus", "ProjectRDC",
        "ProjectYearStarted", "ProjectYearEnded", "ProjectPI"
    ]
    result = enriched[final_cols]
    result["DOI"] = (
        result["DOI"]
        .str.strip()
        .str.replace(r"^https?://(?:dx\.)?doi\.org/", "", regex=True)
    )
    # 6. 保存
    result.to_csv("merged_with_project_metadata.csv", index=False)

    print(f"总记录数：{len(result)}，匹配到项目元数据的记录数：{result['ProjID'].notna().sum()}")

    # 1. 读取原始 CSV
    df = pd.read_csv("merged_with_project_metadata.csv", dtype=str)

    # 2. 对唯一的 DOI 列表去重，避免重复调用
    dois = df["DOI"].dropna().unique().tolist()

    # 3. 批量爬取并收集结果
    records = []
    for doi in dois:
        try:
            print(doi)
            meta = scrape_all_metadata(doi)
            meta["DOI"] = doi
            records.append(meta)
        except Exception as e:
            print(f"爬取 DOI={doi} 时出错：{e}")

    # 4. 构造成 DataFrame
    meta_df = pd.DataFrame.from_records(records)
    # 确保列顺序：DOI 在最前
    cols = ["DOI"] + [c for c in meta_df.columns if c!="DOI"]
    meta_df = meta_df[cols]

    # 5. 与原表按 DOI 合并
    merged = df.merge(meta_df, on="DOI", how="left")

    # 6. 保存结果
    merged.to_csv("with_all_Output_metadata.csv", index=False)
    print("完成，输出保存在 with_all_Output_metadata.csv")

    merged = merged.dropna(subset=["OutputBiblio"])
    merged.to_csv("with_all_Output_metadata_cleaned.csv", index=False)

    df = pd.read_csv("with_all_Output_metadata_cleaned.csv", dtype=str)
    # 4. 逐项过滤
    mask = pd.Series(True, index=df.index)
    mask &= df["OutputPages"].apply(valid_pages)
    # 检查所有列是否包含中文字符
    for column in df.columns:
        # 确保该列是字符串类型
        df[column] = df[column].astype(str)
        mask &= ~df[column].apply(contains_chinese)  # 如果该列含中文字符，则该行被删除
    # 5. 删除列数超过18的行
    mask &= (df.shape[1] <= 18)

    # 5. 应用过滤并查看被删除的数量
    removed = len(df) - mask.sum()
    print(f"将删除 {removed} 行格式不合规或包含中文字符的数据。")

    # 6. 获取清洗后的数据
    df_clean2 = df[mask]

    # 7. 保存结果
    df_clean2.to_csv("with_all_Output_metadata_final.csv", index=False)
    df = pd.read_csv("with_all_Output_metadata_final.csv", dtype=str)

    # 爬取每个 DOI 的作者并打印
    df['author'] = df['DOI'].apply(fetch_author_from_doi)

    # 保存更新后的文件
    df.to_csv("with_all_Output_metadata_with_authors.csv", index=False)

    print("添加作者信息完成，文件已保存为 'with_all_Output_metadata_with_authors.csv'")
    # 重新读取上传的两个文件
    df_authors = pd.read_csv("with_all_Output_metadata_with_authors.csv", dtype=str)
    df_metadata = pd.read_excel("ProjectsAllMetadata.xlsx", sheet_name="Researchers", dtype=str)

    # 提取 Researcher 表中的 Researcher 列和其他需要的列
    researchers = df_metadata[['Researcher', 'Proj ID', 'Status', 'Title', 'RDC', 'Start Year', 'End Year', 'PI']]

    # 定义一个函数来匹配作者并添加相关数据
    def match_researchers_to_project(authors_str):
        # 如果作者为空或NaN，跳过
        if not isinstance(authors_str, str) or not authors_str.strip():
            return []

        # 提取每个作者的姓名（例如：姓, 名）
        authors_list = authors_str.split('; ')
        matched_row = []

        for author in authors_list:
            # 查找 Researcher 表中是否有匹配的作者姓名
            matched = researchers[researchers['Researcher'].str.contains(author, case=False, na=False)]
            if not matched.empty:
                matched_row.append(matched.iloc[0])  # 取第一个匹配的行

        return matched_row

    # 使用 ThreadPoolExecutor 并行化 DOI 爬取，并且添加进度条
    with ThreadPoolExecutor(max_workers=10) as executor:
        # 用 tqdm 包装 iterable 来添加进度条
        matching_data = list(tqdm(executor.map(match_researchers_to_project, df_authors['author']),
                                total=len(df_authors), desc="Processing authors"))

    # 将匹配到的数据添加到 df_authors 中
    proj_columns = ['Proj ID', 'Status', 'Title', 'RDC', 'Start Year', 'End Year', 'PI']
    for col in proj_columns:
        df_authors[col] = [", ".join([str(row[col]) for row in rows]) if rows else "" for rows in matching_data]

    # 保存结果
    df_authors.to_csv("with_all_Output_metadata_with_authors_and_projects.csv", index=False)

    print("数据处理完成，文件已保存为 'with_all_Output_metadata_with_authors_and_projects.csv'")
    # 1. 删除指定列
    columns_to_drop = ['Title', 'ProjID', 'ProjectStatus', 'ProjectRDC', 'ProjectYearEnded', 'ProjectPI']
    df_authors.drop(columns=columns_to_drop, inplace=True)

    # 2. 删除 'Proj ID' 列为空的行
    df_authors.dropna(subset=['Proj ID'], inplace=True)
    df_authors.to_csv("processed_data.csv", index=False)

    # 读取上传的文件
    df = pd.read_csv("processed_data.csv", dtype=str)

    # 删除 'Proj ID' 列为空的行
    df.dropna(subset=['Proj ID'], inplace=True)
    df.drop(columns=['ProjectYearStarted'], inplace=True)
    # 保存处理后的文件
    df.to_csv("processed_data_cleaned.csv", index=False)
    df.to_excel("processed_data_cleaned.xlsx", index=False)
    df.head()

if __name__ == "__main__":
    main()